# Accessing stored vector parts directly

The Query interface is able to return vector-parts based on our needs. This can be useful for numerous reasons:
- testing and monitoring in general
- building logic on the vectors themselves
- and many more.

In [1]:
%pip install superlinked==22.6.3

In [2]:
from superlinked import framework as sl

Let's create a simple config and ingest two entities of text just for demonstration purposes.

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
like_space = sl.NumberSpace(number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM)
# indices can be built on top of multiple spaces as simple as that
paragraph_index = sl.Index([body_space, like_space], fields=paragraph.like_count)

source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

The vector parts are contained in the `Result`'s metadata - it can be switched on and off via the `metadata` argument in any `.select` clause. If you'd like to learn more about `.select` clauses, please refer to the [querying_options notebook](https://github.com/superlinked/superlinked/blob/main/notebook/feature/querying_options.ipynb).

In [4]:
space_weights: dict[sl.Space, float] = {like_space: 1.0, body_space: 1.0}
metadata_query = (
    sl.Query(index=paragraph_index, weights=space_weights).find(paragraph).select_all(metadata=[body_space, like_space])
)

result = app.query(metadata_query)

Note, that this is a [Result](https://github.com/superlinked/superlinked/blob/main/notebook/feature/query_result.ipynb) (coming from the same [query interface](https://github.com/superlinked/superlinked/blob/main/notebook/feature/querying_options.ipynb), just like any other - just with additional metadata.

In [5]:
first_entry = result.entries[0]
second_entry = result.entries[1]

For the first entry, let's examine the `vector_parts` attribute of the metadata object. This is where our vectors are going to be.

The first vector part is a 768 long text embedding of the `body attribute` - that is in line with the dimensionality of the embedding of the [model](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) used.

In [6]:
print(len(first_entry.metadata.vector_parts[0]))
first_entry.metadata.vector_parts[0][:3]

768


[0.012647623288110286, 0.05507237348049912, -0.023800908901327893]

The second vector part is our very own [number embedding](https://github.com/superlinked/superlinked/blob/main/notebook/feature/number_embedding_minmax.ipynb) of the `like` attribute.

In [7]:
first_entry.metadata.vector_parts[1]

[0.6532814824381882, 0.2705980500730985, 0.0]

The same structure applies to the second entry in the `Result` as well.

In [8]:
print(len(second_entry.metadata.vector_parts[0]))
second_entry.metadata.vector_parts[0][:3]

768


[-0.012543628009221226, 0.055735641262102856, -0.034339140038615434]

In [9]:
second_entry.metadata.vector_parts[1]

[0.11061587104123714, 0.6984011233337103, 0.0]

<div class="alert alert-block alert-info">
⚠️ Keep in mind that the vector part order in the metadata is <b>ALWAYS</b> following the space order when the index was created.
</div>



In [10]:
alternative_order_query = (
    sl.Query(index=paragraph_index, weights=space_weights).find(paragraph).select_all(metadata=[like_space, body_space])
)  # switched order of the spaces

result_alternative = app.query(alternative_order_query)

The space order in the vector parts inside metadata is unaffected by the order in which the input was supplied to the `metadata` argument of the `.select_all()` clause.

In [11]:
result_alternative.entries[0].metadata.vector_parts[0][:3] == first_entry.metadata.vector_parts[0][:3]

True